In [98]:
import csv
import numpy as np
from tabulate import tabulate
from itertools import product
from collections import defaultdict

%cd /storage2/mwildi/CBS2
#####################################################################################
# Parsing code adapted from World on Rails scripts/view_nocrash_results
#####################################################################################

def parse_results(path, traffic=None, metric_out=None):
    TOWNS = ['Town01', 'Town02']
    TRAFFICS = ['empty', 'regular', 'dense']
    WEATHERS = {
        1: 'train', 3: 'train', 6: 'train', 8: 'train',
        10: 'test', 14: 'test',
    }
    METRICS = ['Success Rate', 'Route Completion', 'Lights Ran', 'No Block', 'No Collision', 'No Collision Veh', 'No Collision Walk']

    if traffic=='empty':
        TRAFFICS = ['empty']
    elif traffic=='regular':
        TRAFFICS = ['regular']
    elif traffic=='dense':
        TRAFFICS = ['dense']

    finished = defaultdict(lambda: [])

    with open(path+'.csv', 'r') as file:
        log = csv.DictReader(file)
        for row in log:
            finished[(
                row['town'],
                int(row['traffic']),
                WEATHERS[int(row['weather'])],
            )].append((
                float(row['route_completion']),
                int(row['lights_ran']),
                float(row['duration']),
                float(row['outside_lane']),
                str(row['collision']),
                str(row['collision_vehicle']),
                str(row['collision_walker']),
                str(row['in_route']),
                str(row['blocked'])
            ))

    all_stats=dict()
    
    for town, weather_set in product(TOWNS, set(WEATHERS.values())):

        output = "\n"
        output += "\033[1m========= Results of {}, weather {} \033[1m=========\033[0m\n".format(
                town, weather_set)
        output += "\n"
        list_statistics = [['Traffic', *TRAFFICS], [metric_out]+['N/A']*len(TRAFFICS),['Duration']+['N/A']*len(TRAFFICS)]

        all_stats[town+weather_set] = {}
        
        for metric in METRICS:
            stat= ['N/A']*len(TRAFFICS)

            for traffic_idx, traffic in enumerate(TRAFFICS):
                runs = finished[town,TRAFFICS.index(traffic),weather_set]

                if len(runs) > 0:
                    route_completion, lights_ran, duration, outside_lane, collision, collision_vehicle, collision_walker, in_route, blocked = zip(*runs)
                    mean_lights_ran = np.array(lights_ran)/np.array(duration)*3600

                    if metric == 'Success Rate':
                        stat[traffic_idx] = np.round(100*np.mean(np.array(route_completion)==100), 2)
                    elif metric == 'Route Completion':
                        stat[traffic_idx] = np.round(np.mean(route_completion), 2)
                    elif metric == 'Lights Ran ph':
                        stat[traffic_idx] = np.round(np.mean(mean_lights_ran), 2)
                    elif metric == 'No Block':
                        stat[traffic_idx] = np.round(100*blocked.count('SUCCESS')/len(blocked), 2)
                    elif metric == 'No Collision':
                        stat[traffic_idx] = round(100*collision.count('SUCCESS')/len(collision), 2)
                    elif metric == 'No Collision Veh':
                        stat[traffic_idx] = np.round(100*collision_vehicle.count('SUCCESS')/len(collision_vehicle), 2)
                    elif metric == 'No Collision Walk':
                        stat[traffic_idx] = np.round(100*collision_walker.count('SUCCESS')/len(collision_walker), 2)

                    if metric == metric_out:
                        list_statistics[1][traffic_idx+1] = "{}%".format(stat[traffic_idx])
                    list_statistics[2][traffic_idx+1] = "{}s".format(round(np.mean(duration), 2))
                    
                all_stats[town+weather_set][metric] = stat
        output += tabulate(list_statistics, tablefmt='fancy_grid')
        output += "\n"
        if metric_out is not None:
            print(output)
    return all_stats

/storage2/mwildi/CBS2


In [101]:
d=parse_results('results/final/test_original')

In [102]:
d

{'Town01test': {'Success Rate': ['N/A', 'N/A', 'N/A'],
  'Route Completion': ['N/A', 'N/A', 'N/A'],
  'Lights Ran': ['N/A', 'N/A', 'N/A'],
  'No Block': ['N/A', 'N/A', 'N/A'],
  'No Collision': ['N/A', 'N/A', 'N/A'],
  'No Collision Veh': ['N/A', 'N/A', 'N/A'],
  'No Collision Walk': ['N/A', 'N/A', 'N/A']},
 'Town01train': {'Success Rate': ['N/A', 'N/A', 'N/A'],
  'Route Completion': ['N/A', 'N/A', 'N/A'],
  'Lights Ran': ['N/A', 'N/A', 'N/A'],
  'No Block': ['N/A', 'N/A', 'N/A'],
  'No Collision': ['N/A', 'N/A', 'N/A'],
  'No Collision Veh': ['N/A', 'N/A', 'N/A'],
  'No Collision Walk': ['N/A', 'N/A', 'N/A']},
 'Town02test': {'Success Rate': [20.0, 28.0, 12.0],
  'Route Completion': [51.41, 67.74, 50.97],
  'Lights Ran': ['N/A', 'N/A', 'N/A'],
  'No Block': [52.0, 96.0, 98.0],
  'No Collision': [84.0, 54.0, 22.0],
  'No Collision Veh': [100.0, 82.0, 44.0],
  'No Collision Walk': [100.0, 100.0, 100.0]},
 'Town02train': {'Success Rate': ['N/A', 'N/A', 'N/A'],
  'Route Completion': ['N/A